In [1]:
'''
This file uses the experimental tool published by tensorflow to load
tiff files

I expect the subset ot look like:
foldername:
        label1:
            img1
            img2
            img3 
            ...
        ...

'''

import pandas as pd
import tensorflow as tf
import numpy as np
import json
import tensorflow_io as tfio
import os

from load_tiff import process_path


class process_tiff_img_set():
    def __init__(self, ds_folder_name):
        
        self.folder = ds_folder_name
        self.df =  self.create_df()
        #self.ds = self.load_from_df()
        self.ds = self.df

       

    def create_df(self):
        '''
        This function creates a dataframe with all labels and 
        images file paths
        I will use later to store numpy arrays
        '''
        ds = {}
        DS_PATH = os.path.join(os.getcwd(), self.folder)
        labels = os.listdir(DS_PATH)
        for lbl in labels:
            cwd = os.path.join(DS_PATH, lbl)
            if(not os.path.isdir(cwd)):
                print(f'{cwd} is not a directory. Will not be in dict')
                continue
            images = os.listdir(cwd)
            for img in images:
                #convert to numpy matrix here
                path = os.path.join(cwd, img)
                img_np = process_path(path) #Tensor 
                if(lbl in ds):
                    #print(f'#ds[{lbl} = [..., {os.path.join(cwd, img)}]')
                    
                    ds[lbl].append(img_np)
                else:
                    #print(f'*ds[{lbl} = [{os.path.join(cwd, img)}]]')
                    ds[lbl] = []
                    ds[lbl].append(img_np)
        df = pd.DataFrame.from_dict(ds)
        df.to_csv('ds.csv')
        return df
        
    def get_dataframe(self):
        '''Create a datrame from json file'''
        file = open('ds.json', 'r')
        df = pd.DataFrame(json.load(file))
        file.close()
        return df

    def decode_img(self, img):

        
        #print(f'decoding ... ')
        img = tfio.experimental.image.decode_tiff(img, index=0, name=None)

        # resize the image to the desired size
        return tf.image.resize(img, [256, 256])


    def process_path(self, file_path):

        # load the raw data from the file as a string
        #print(f'reading {file_path}')
        img = tf.io.read_file(file_path)
        img = self.decode_img(img)

        return img


    def load_from_df(self):
        '''
        This function loads a tensorflow dataset from pandas dataframe
        WARNINGL: This is for expermentation, edit code to automize
        '''
        df = self.df
        if(df.empty):
            #print(f'df is empty, loading existing df')
            df = pd.read_csv('ds.csv')
        self.ds = {}
        for date in df:
            for i, img in enumerate(date):
                tensor = tf.convert_to_tensor(img)
                if(date not in self.ds):
                    self.ds[date] = []
                self.ds[date].append(tensor)
        pd.DataFrame(self.ds).to_csv('processed_ds.csv')
        return self.ds

    def get_tensor(self):
        return pd.DataFrame(self.ds)
    def get_dataframe(self):
        return self.df

    def __str__(self):
        obj_str = ""
        for i, obj in enumerate(self.ds):
            obj_str += f'{i}: {str(obj)},\n'
        return f'<process_tiff_img_set>:\n{obj_str}'



In [2]:
'''
This file is influeced by Tensorflow tutorial
More edits will be applied to it

This file contains the implemetation of the neural networks
For now it is very simple
Encoder: Flatten -> Dense
Decoder: Dense- ->
'''
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from load_imgs import load_dataset

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model



class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28, 28))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


In [4]:
  
from autoencoder import *
from process_tiff_img_set import process_tiff_img_set

ds = process_tiff_img_set("./subset")
ds = ds.get_tensor()
print(f'tensor shape: {ds.shape}')
#print(f'{ds}')



autoencoder = Autoencoder(64) 

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.fit(ds, ds,
                epochs=1,
                shuffle=True,
                validation_data=(ds, ds), verbose='auto')
autoencoder.save('saved_models/autoencoder_0')

/Users/ualguest/dev/autoencoder/./subset/.DS_Store is not a directory. Will not be in dict
reading /Users/ualguest/dev/autoencoder/./subset/2020-06-23__10-36-48-868/2020-06-23__10-36-48-868___0901___BTx_623.tif
decoding ... 


TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.


reading /Users/ualguest/dev/autoencoder/./subset/2020-06-23__10-36-48-868/2020-06-23__10-36-48-868___0931___PI_678004.tif
decoding ... 
reading /Users/ualguest/dev/autoencoder/./subset/2020-06-23__10-36-48-868/2020-06-23__10-36-48-868___0729___PI_678040.tif
decoding ... 


TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.


reading /Users/ualguest/dev/autoencoder/./subset/2020-06-23__10-36-48-868/2020-06-23__10-36-48-868___0801___BTx_623.tif
decoding ... 


...

TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.


reading /Users/ualguest/dev/autoencoder/./subset/2020-06-23__10-36-48-868/2020-06-23__10-36-48-868___0725___PI_678118.tif
decoding ... 


TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 42113 (0xa481) encountered.


tensor shape: (7, 1)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type tensorflow.python.framework.ops.EagerTensor).